In [1]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_squared_error
import numpy as np
import os

# Load data
csv_path = 'all_temperature_cleaned.csv'
df = pd.read_csv(csv_path)
df.columns = df.columns.str.strip()
df['datetime'] = pd.to_datetime(df['timestamp'] + ' ' + df['time'], format='%Y-%m-%d %H:%M')

regions = ['Rakhiyal', 'Bopal', 'Ambawadi', 'Chandkheda', 'Vastral']

# Split data into train (2019-2022), validation (2023), test (2024)
train_df = df[(df['datetime'].dt.year >= 2019) & (df['datetime'].dt.year <= 2022)].copy()
valid_df = df[df['datetime'].dt.year == 2023].copy()
test_df = df[df['datetime'].dt.year == 2024].copy()

# Output folders
output_folder_2024 = 'trial/predictions_2024'
output_folder_2025 = 'trial/predictions_2025'
os.makedirs(output_folder_2024, exist_ok=True)
os.makedirs(output_folder_2025, exist_ok=True)

current_dir = os.getcwd()
metrics_list = []

# Define parameter grid for tuning
changepoint_prior_scales = [0.001, 0.01, 0.1, 0.5]
seasonality_prior_scales = [0.01, 0.1, 1.0]
seasonality_modes = ['additive', 'multiplicative']

for region in regions:
    print(f"\nTuning Prophet parameters for region: {region}")

    # Prepare train and validation data for Prophet
    train_prophet = train_df[['datetime', region]].rename(columns={'datetime': 'ds', region: 'y'})
    valid_prophet = valid_df[['datetime', region]].rename(columns={'datetime': 'ds', region: 'y'})

    best_rmse = float('inf')
    best_params = None
    best_model = None

    # Grid search over parameters
    for cps in changepoint_prior_scales:
        for sps in seasonality_prior_scales:
            for smode in seasonality_modes:
                try:
                    model = Prophet(
                        changepoint_prior_scale=cps,
                        seasonality_prior_scale=sps,
                        seasonality_mode=smode,
                        daily_seasonality=True,
                        yearly_seasonality=True,
                        weekly_seasonality=True
                    )
                    model.fit(train_prophet)

                    # Predict on validation set
                    future_valid = valid_prophet[['ds']].copy()
                    forecast_valid = model.predict(future_valid)

                    y_valid_true = valid_prophet['y'].values
                    y_valid_pred = forecast_valid['yhat'].values

                    rmse = np.sqrt(mean_squared_error(y_valid_true, y_valid_pred))

                    if rmse < best_rmse:
                        best_rmse = rmse
                        best_params = {
                            'changepoint_prior_scale': cps,
                            'seasonality_prior_scale': sps,
                            'seasonality_mode': smode
                        }
                        best_model = model
                except Exception as e:
                    print(f"  Skipping params cps={cps}, sps={sps}, mode={smode} due to error: {e}")
                    continue

    print(f"Best params for {region}: {best_params} with validation RMSE: {best_rmse:.4f}")

    # Refit best model on train + validation data
    combined_df = pd.concat([train_prophet, valid_prophet])
    final_model = Prophet(
        changepoint_prior_scale=best_params['changepoint_prior_scale'],
        seasonality_prior_scale=best_params['seasonality_prior_scale'],
        seasonality_mode=best_params['seasonality_mode'],
        daily_seasonality=True,
        yearly_seasonality=True,
        weekly_seasonality=True
    )
    final_model.fit(combined_df)

    # Predict on 2024 test data
    test_prophet = test_df[['datetime', region]].rename(columns={'datetime': 'ds', region: 'y'})
    future_test = test_prophet[['ds']].copy()
    forecast_test = final_model.predict(future_test)

    y_test_true = test_prophet['y'].values
    y_test_pred = forecast_test['yhat'].values
    pred_dates = test_prophet['ds']

    mse_test = mean_squared_error(y_test_true, y_test_pred)
    rmse_test = np.sqrt(mse_test)
    r2_test = 1 - np.sum((y_test_true - y_test_pred) ** 2) / np.sum((y_test_true - np.mean(y_test_true)) ** 2)

    print(f"  2024 Test MSE: {mse_test:.4f}")
    print(f"  2024 Test RMSE: {rmse_test:.4f}")
    print(f"  2024 Test R2: {r2_test:.4f}")

    # Save 2024 predictions CSV (date/hour format)
    pred_2024_df = pd.DataFrame({
        'date': pred_dates.dt.date,
        'hour': pred_dates.dt.hour,
        'predicted_temperature': y_test_pred,
        'actual_temperature': y_test_true
    })
    filename_2024 = f"prophet_{region.lower()}_2024.csv"
    pred_2024_df.to_csv(os.path.join(output_folder_2024, filename_2024), index=False)
    print(f"Saved 2024 Prophet predictions for {region} as {filename_2024}")

    # Forecast 2025 hourly
    last_date = combined_df['ds'].max()
    future_end = pd.Timestamp('2025-12-31 23:00:00')
    n_hours = int((future_end - last_date).total_seconds() // 3600)

    future_2025 = final_model.make_future_dataframe(periods=n_hours, freq='h')
    future_2025 = future_2025[future_2025['ds'] >= '2025-01-01']

    forecast_2025 = final_model.predict(future_2025)

    pred_2025_df = pd.DataFrame({
        'date': forecast_2025['ds'].dt.date,
        'hour': forecast_2025['ds'].dt.hour,
        'predicted_temperature': forecast_2025['yhat']
    })

    filename_2025 = f"prophet_{region.lower()}_2025.csv"
    pred_2025_df.to_csv(os.path.join(output_folder_2025, filename_2025), index=False)
    print(f"Saved 2025 Prophet predictions for {region} as {os.path.join(output_folder_2025, filename_2025)}")

    # Store metrics
    metrics_list.append({
        'region': region,
        'mse_2024': mse_test,
        'rmse_2024': rmse_test,
        'r2_2024': r2_test,
        'best_params': best_params
    })

# Save metrics CSV
metrics_df = pd.DataFrame(metrics_list)
metrics_filename = 'prophet_auto_tuned_metrics_2024.csv'
metrics_df.to_csv(os.path.join(current_dir, metrics_filename), index=False)
print(f"Saved auto-tuned Prophet metrics for all regions as {metrics_filename}")


C:\Users\PARIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Tuning Prophet parameters for region: Rakhiyal


06:15:34 - cmdstanpy - INFO - Chain [1] start processing
06:15:38 - cmdstanpy - INFO - Chain [1] done processing
06:15:43 - cmdstanpy - INFO - Chain [1] start processing
06:15:47 - cmdstanpy - INFO - Chain [1] done processing
06:15:52 - cmdstanpy - INFO - Chain [1] start processing
06:15:55 - cmdstanpy - INFO - Chain [1] done processing
06:16:00 - cmdstanpy - INFO - Chain [1] start processing
06:16:04 - cmdstanpy - INFO - Chain [1] done processing
06:16:09 - cmdstanpy - INFO - Chain [1] start processing
06:16:15 - cmdstanpy - INFO - Chain [1] done processing
06:16:20 - cmdstanpy - INFO - Chain [1] start processing
06:16:24 - cmdstanpy - INFO - Chain [1] done processing
06:16:29 - cmdstanpy - INFO - Chain [1] start processing
06:16:40 - cmdstanpy - INFO - Chain [1] done processing
06:16:45 - cmdstanpy - INFO - Chain [1] start processing
06:16:54 - cmdstanpy - INFO - Chain [1] done processing
06:16:59 - cmdstanpy - INFO - Chain [1] start processing
06:17:10 - cmdstanpy - INFO - Chain [1]

Best params for Rakhiyal: {'changepoint_prior_scale': 0.001, 'seasonality_prior_scale': 0.01, 'seasonality_mode': 'multiplicative'} with validation RMSE: 2.0782


06:28:48 - cmdstanpy - INFO - Chain [1] start processing
06:28:55 - cmdstanpy - INFO - Chain [1] done processing


...

  2024 Test MSE: 4.9095
  2024 Test RMSE: 2.2157
  2024 Test R2: 0.8644
Saved 2024 Prophet predictions for Chandkheda as prophet_chandkheda_2024.csv
Saved 2025 Prophet predictions for Chandkheda as trial/predictions_2025\prophet_chandkheda_2025.csv

Tuning Prophet parameters for region: Vastral


11:59:30 - cmdstanpy - INFO - Chain [1] start processing
11:59:34 - cmdstanpy - INFO - Chain [1] done processing
11:59:38 - cmdstanpy - INFO - Chain [1] start processing
11:59:44 - cmdstanpy - INFO - Chain [1] done processing
11:59:49 - cmdstanpy - INFO - Chain [1] start processing
11:59:52 - cmdstanpy - INFO - Chain [1] done processing
11:59:57 - cmdstanpy - INFO - Chain [1] start processing
12:00:00 - cmdstanpy - INFO - Chain [1] done processing
12:00:05 - cmdstanpy - INFO - Chain [1] start processing
12:00:09 - cmdstanpy - INFO - Chain [1] done processing
12:00:14 - cmdstanpy - INFO - Chain [1] start processing
12:00:18 - cmdstanpy - INFO - Chain [1] done processing
12:00:23 - cmdstanpy - INFO - Chain [1] start processing
12:00:32 - cmdstanpy - INFO - Chain [1] done processing
12:00:37 - cmdstanpy - INFO - Chain [1] start processing
12:00:46 - cmdstanpy - INFO - Chain [1] done processing
12:00:51 - cmdstanpy - INFO - Chain [1] start processing
12:01:01 - cmdstanpy - INFO - Chain [1]

Best params for Vastral: {'changepoint_prior_scale': 0.001, 'seasonality_prior_scale': 0.01, 'seasonality_mode': 'multiplicative'} with validation RMSE: 2.0814


12:09:09 - cmdstanpy - INFO - Chain [1] start processing
12:09:17 - cmdstanpy - INFO - Chain [1] done processing


  2024 Test MSE: 4.8597
  2024 Test RMSE: 2.2045
  2024 Test R2: 0.8616
Saved 2024 Prophet predictions for Vastral as prophet_vastral_2024.csv
Saved 2025 Prophet predictions for Vastral as trial/predictions_2025\prophet_vastral_2025.csv
Saved auto-tuned Prophet metrics for all regions as prophet_auto_tuned_metrics_2024.csv
